## Stochastic Programming Example Using StochJump and DSP

## Farmer Crop Optimization

## Model Data

#### Here, we setup the data relevant to the farmer model.  We set 3 possible scenarios of equal probability, and then three sets of crops to plant with the associated costs of planting each crop.  We also define the total budget.  The second stage model data adds the possiblity to purchase and sell crops with corresponding purchase and sell prices.

In [1]:
# Farmer example from Birge and Louveaux book.

# STOCHASTIC MODELING FRAMEWORK

NS = 3;                        # number of scenarios
probability = [1/3, 1/3, 1/3]; # probability


# FIRST-STAGE MODEL
CROPS = 1:3; # set of crops (wheat, corn and sugar beets, resp.)
Cost = [150 230 260]; # cost of planting crops
Budget = 500; # budget capacity


# SECOND-STAGE MODELS
PURCH = 1:2; # set of crops to purchase (wheat and corn, resp.)
SELL  = 1:4; # set of crops to sell (wheat, corn, sugar beets under 6K and those over 6K)
Purchase = [238 210;
            238 210;
            238 210];   # purchase price
Sell = [170 150 36 10;
        170 150 36 10;
        170 150 36 10]; # selling price
Yield = [3.0 3.6 24.0;
         2.5 3.0 20.0;
         2.0 2.4 16.0];
Minreq = [200 240 0;
          200 240 0;
          200 240 0]; # minimum crop requirement
println("")

## Model Expression

#### Using the actual data defined above, we express the model using StochJuMP macros in terms of first and second stage variables.

In [2]:
using MPI, StochJuMP, DSPsolver
# CREATE STOCHASTIC MODEL
ENV["LD_LIBRARY_PATH"] = "DSP/lib"
m = StochasticModel(NS);


# FIRST-STAGE MODEL

# first-stage variables
@defVar(m, x[i=CROPS] >= 0, Int)

# first-stage objective
@setObjective(m, Min, sum{Cost[i] * x[i], i=CROPS})

# first-stage constraint
@addConstraint(m, const_budget,
               sum{x[i], i=CROPS} <= Budget)


# SECOND-STAGE MODELS

@second_stage m s begin
    # stochastic block
    sb = StochasticBlock(m, probability[s]);

    # second-stage variables
    @defVar(sb, y[j=PURCH] >= 0)
    @defVar(sb, w[k=SELL] >= 0)

    # objective
    @setObjective(sb, Min,
                  sum{Purchase[s,j] * y[j], j=PURCH}
                  - sum{Sell[s,k] * w[k], k=SELL})

    # constraints
    @addConstraint(sb, const_minreq[j=PURCH],
                   Yield[s,j] * x[j] + y[j] - w[j] >= Minreq[s,j])
    @addConstraint(sb, const_minreq_beets,
                   Yield[s,3] * x[3] - w[3] - w[4] >= Minreq[s,3])
    @addConstraint(sb, const_aux, w[3] <= 6000)
end

print(m)

Min 150 x[1] + 230 x[2] + 260 x[3]
Subject to
 x[1] + x[2] + x[3] ≤ 500
 x[i] ≥ 0, integer, ∀ i ∈ {1,2,3}


#### Using StochJuMP, we have produced the complete optimization problem in standard form

# Solving With DSP

#### Finally, We use the DSP Solver to solve the optimization problem and output the results

In [3]:
option = "BD"

if option == "DD"
	MPI.Init();
end

# data file
#include("farmer_data.jl")

# model file
#include("farmer_model.jl")

# load problem to model object
DSPsolver.loadProblem(m);

# set parameters
DSPsolver.setIntParam("LOG_LEVEL",1);
DSPsolver.setIntParam("ITER_LIM",100);
DSPsolver.setDblParam("SCIP/GAP_TOL",0.0);

# solve problem
if option == "DE"
	DSPsolver.solve(DSP_SOLVER_DE);
elseif option == "BD"
	DSPsolver.setIntParam("BD/NUM_CORES",1);
	DSPsolver.solve(DSP_SOLVER_BD);
elseif option == "DD"
	DSPsolver.setIntParam("DD/FEAS_CUTS",1);
	DSPsolver.setIntParam("DD/OPT_CUTS",1);
	DSPsolver.setIntParam("DD/EVAL_UB",1);
	DSPsolver.setIntParam("DD/MASTER_ALGO",IPM_FEAS);
	DSPsolver.solve(DSP_SOLVER_DD);
end

if MPI.Initialized() == false || MPI.Comm_rank(MPI.COMM_WORLD) == 0
	# print some results
	println("Solution Status: ", DSPsolver.getSolutionStatus());
	println("Primal Bound   : ", DSPsolver.getPrimalBound());
	println("Dual Bound     : ", DSPsolver.getDualBound());
end

if option == "DD"
	MPI.Finalize();
end


Phase 1:
Creating Benders sub problems ... (0.00 sec)
Finding global lower bound ... (0.00 sec) -> Lower bound -1.154056e+05
Creating master problem instance ... (0.01 sec)

Phase 2:



presolving (2 rounds):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 4 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 4 variables (0 bin, 3 int, 0 impl, 1 cont) and 3 constraints

 time | node  | left  |LP iter|LP it/n| mem |mdpt |frac |vars |cons |cols |rows |cuts |confs|strbr|  dualbound   | primalbound  |  gap   
* 0.0s|     1 |     0 |    21 |     - | 187k|   0 |   - |   4 |   3 |   4 |  15 |  14 |   0 |   0 |-1.083900e+05 |-1.083900e+05 |   0.00%

SCIP Status        : problem is solved [optimal solution found]
Solving Time (sec) : 0.01
Solving Nodes      : 1
Primal Bound       : -1.08390000000000e+05 (1 solutions)
Dual Bound         : -1.08390000000000e+05
Gap                : 0.00 %
Solution Status: Optimal
Primal Bound   : -108389.99999999999
Dual Bound     : -108390.00000000003
